### [Merge K sorted lists](https://leetcode.com/problems/merge-k-sorted-lists/)

Merge k sorted linked lists and return it as one sorted list. Analyze and describe its complexity.

Example:

```
Input:
[
  1->4->5,
  1->3->4,
  2->6
]
```

Output: 1->1->2->3->4->4->5->6

In [2]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [9]:
import heapq
from queue import PriorityQueue
from typing import List

class Solution:
    # trying using heap (applies to priority queue as well)
    #   maintain a min heap of length k
    #   push the first item from each list into the heap
    #   x = heappop(heap)
    #   connect x to the tail
    #   if x has a next:
    #       heappush(heap, x.next)
    #
    # that is a push and a pop for each element
    #   each push/pop takes log(k) time
    #   2(kn) * log(k) -> kn(log(k)) - this is faster than knlog(nk)
    
    def mergeKLists(self, lists: List[ListNode]) -> ListNode:
        
        if not lists:
            return None
        
        k = len(lists)
        
        if k == 1:
            return lists[0]

        queue = PriorityQueue()
        head = tail = ListNode(-1)
        node_id = 0
        
        # first add first column elements into the heap
        for node in filter(None, lists):
            queue.put((node.val, node_id, node))
            node_id += 1
        
        # work until the heap is empty
        while not queue.empty():
            _, _, node = queue.get()
            
            if node.next:
                queue.put((node.next.val, node_id, node.next))
                node_id += 1
                
            tail.next = node
            tail = tail.next
            
        return head.next
    
    def mergeKListsUsingHeap(self, lists: List[ListNode]) -> ListNode:
        
        if not lists:
            return None
        
        k = len(lists)
        
        if k == 1:
            return lists[0]

        heap = []
        head = tail = ListNode(-1)
        node_id = 0
        
        # first add first column elements into the heap
        for node in lists:
            if node:
                heap.append((node.val, node_id, node))
                node_id += 1
        
        heapq.heapify(heap)

        # work until the heap is empty
        while heap:
            _, _, next_node = heapq.heappop(heap)
            
            if next_node.next:
                heapq.heappush(heap, (next_node.next.val, node_id, next_node.next))
                node_id += 1
                
            tail.next = next_node
            tail = tail.next
            
        return head.next
        
        
    def _merge(self, list1, list2):
        # if one of them is empty
        # return the other
        if not list1:
            return list2

        if not list2:
            return list1

        # have a dummy head, tail
        head = tail = ListNode(-1)

        l1, l2 = list1, list2
        
        # we iterate list1, list2
        # pick min(l1, l2)
        # tail.next = min(l1, l2)
        # tail = tail.next
        # l1 = l1.next
        # l2 = l2.next

        # finally add the remaining ones in l1 and l2 separately.

        while l1 and l2:
            tail.next = min(l1, l2, key=lambda x: x.val)

            if tail.next == l1:
                l1 = l1.next
            else:
                l2 = l2.next

            tail = tail.next

        # we don't need a while loop.
        # since this is list, we could simply connect them
        if l1:
            tail.next = l1
        
        if l2:
            tail.next = l2

        return head.next

    def mergeKListsDC(self, lists: List[ListNode]) -> ListNode:
        # list of sorted linked lists.
        # merge them into one.
        
        # merge l1,l2
        # merge (l1, l2) with l3
        # that will be expensive 
        
        # 1-4-5
        # 1-3-4
        # 2-4
        
        # 2n + 3n + 4n... m*n
        # n (m^2) => O(n^3), if m == n
        
        #   (1, 2) (3, 4) (5, 6) (7, 8)
        #    (1, 2, 3, 4) (5, 6, 7, 8)
        #    (1, 2, 3, 4, 5, 6, 7, 8)
        
        if not lists:
            return None
        
        if len(lists) == 1:
            return lists[0]
        
        # to treat this merge sort. 
        # l1 = mergeKLists(0..K/2)
        # l2 = mergeKLists(K/2..K)
        # l3 = merge(l1, l2)
        k = len(lists)
        l1 = self.mergeKLists(lists[:k//2])
        l2 = self.mergeKLists(lists[k//2:])
        
        l3 = self._merge(l1, l2)
        
        return l3

In [24]:
# Some tests for functional correctness
tests = {
    "test": [
        {
            "input": [
                [1,4,5],
                [1,3,4],
                [2,6]
            ],
            "output": [1,1,2,3,4,4,5,6]
        },
        {
            "input": [
                [1,4,5],
                [1,3,4],
                [2,6],
                [0],
                [0, 1, 2]
            ],
            "output": [0,0,1,1,1,2,2,3,4,4,5,6]
        },
        {
            "input": [[]],
            "output": []
        }
    ]
        
    
}

In [11]:
def list_to_linked_list(nums):
    head = tail = ListNode(-1)
    for num in nums:
        tail.next = ListNode(num)
        tail = tail.next
        
    return head.next

def linked_list_to_list(head):
    nums = []
    while head:
        nums.append(head.val)
        head = head.next
    return nums

In [22]:
def runTests(tests):
    s = Solution()
    for test in tests["test"]:
        sorted_lists, output = test["input"], test["output"]
        
        # have to generate new lists every solution because
        # we merge them in the processing. can't reuse them directly
        sorted_linked_lists = [list_to_linked_list(sorted_list) for sorted_list in sorted_lists]
        merged_linked_list = s.mergeKLists(sorted_linked_lists)
        assert(linked_list_to_list(merged_linked_list) == output)
        
        sorted_linked_lists = [list_to_linked_list(sorted_list) for sorted_list in sorted_lists]
        merged_linked_list = s.mergeKListsUsingHeap(sorted_linked_lists)
        assert(linked_list_to_list(merged_linked_list) == output)
        
        sorted_linked_lists = [list_to_linked_list(sorted_list) for sorted_list in sorted_lists]
        merged_linked_list = s.mergeKListsDC(sorted_linked_lists)
        assert(linked_list_to_list(merged_linked_list) == output)

In [23]:
runTests(tests)